# Como usar:

1. Criar um diretorio raiz chamado Skps;
2. Criar um subdiretório que armazenará as imagens;
3. Mover as imagens para o subdiretório;
4. Adicionar em locals o nome do diretorio onde as imagens dos skp's estão armazenadas;
5. Definir no campo variáveis os nomes de agencia e nome;


Pontos de Atencão: é muito importante a foto do canhoto ser de qualidade e não haver nenhum faltando na sequencia fisica.

Labels de nomes devem ser adicionados aos nomes das imagens.



# install


In [ ]:
# Install the zbar shared library
!apt-get update
!apt-get install -y libzbar0
!pip install pyzbar

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

# LibsPy

In [ ]:
from pyzbar.pyzbar import decode
from PIL import Image
import pandas as pd
import os
from datetime import datetime
from google.colab import userdata

# Locals

In [ ]:
# local dos images skps
os.chdir(f"{userdata.get('base')}AgenciaTeste")

# Funções

In [ ]:
from typing import List
def ler_codigo_de_barras(image_path):
    try:
        img = Image.open(image_path)
        decoded_objects = decode(img)
        if decoded_objects:
            return [obj.data.decode('utf-8') for obj in decoded_objects]
        else:
            return None
    except FileNotFoundError:
        print(f"Erro: Arquivo não encontrado em {image_path}")
        return None
    except Exception as e:
        print(f"Erro ao decodificar a imagem: {e}")
        return None

def maior_menor(skp:List):
  aux = []
  for item in skp:
    aux.append(int(item.replace('T', '')))
  return [min(aux), max(aux)]

def sequencia(skp:List):
  return [skps for skps in range(skp[0], skp[1]+1)]

def tipagem():
  res = []
  files = os.listdir()
  for file in files:
    if file.endswith('_B.jpg'):
      res.append('Tipo B')
    elif file.endswith('_A.jpg'):
      res.append('Tipo A')
    elif file.endswith('_C.jpg'):
      res.append('Tipo C')
  return res

# Total de Imagens

In [ ]:
print('Total de Imagens')
len(os.listdir())

Total de Imagens


3

# Execução

In [ ]:
skp = []
for image in os.listdir():
  skp.append(ler_codigo_de_barras(image))
  print('imagem: ', image)

imagem:  Primeiro_B.jpg
imagem:  Segundo_B.jpg
imagem:  Terceiro_A.jpg


In [ ]:
skp

[['T2038981738', 'T2038981597'],
 ['T2038981596', 'T2038981559'],
 ['T2044619583', 'T2044619563']]

In [ ]:
lista_skp = [maior_menor(item) for item in skp]

# Canhotos por Sequencia

In [ ]:
# Tamanho dos Itens Sequencia
total = []
for i, item in enumerate(skp):
  seq = sequencia(maior_menor(item))
  print(len(seq))
  lista_skp[i].append(len(seq))
  lista_skp[i].append(tipagem()[i])
  total.append(len(seq))
print('Total de Caixas prontas: ', sum(total))

142
38
21
Total de Caixas prontas:  201


In [ ]:
#cols_relatorio = ['Inicial e Final', 'Sequencia', 'Tipo', 'Quantidade']
cols_relatorio = ['Inicial', 'Final', 'Quantidade', 'Tipo']

In [ ]:
lista_skp

[[2038981597, 2038981738, 142, 'Tipo B'],
 [2038981559, 2038981596, 38, 'Tipo B'],
 [2044619563, 2044619583, 21, 'Tipo A']]

In [ ]:
df = pd.DataFrame(lista_skp, columns=cols_relatorio)

# Variáveis

In [ ]:
cgc  = '001'
nome = 'Agteste'
df['Agencia'] = cgc
df['Nome'] = nome

In [ ]:
df

,Inicial,Final,Quantidade,Tipo,Agencia,Nome
0,2038981597,2038981738,142,Tipo B,001,Agteste
1,2038981559,2038981596,38,Tipo B,001,Agteste
2,2044619563,2044619583,21,Tipo A,001,Agteste


In [ ]:
# Caixas prontas
Relatorio = f'''

\033[1m# Relatório Agencia {nome} - CGC: {agencia}\033[0m

\033[1mTipo A: {df[df['Tipo'] == 'Tipo A']['Quantidade'].sum()}\033[0m
\033[1mTipo B: {df[df['Tipo'] == 'Tipo B']['Quantidade'].sum()}\033[0m
\033[1mTipo C: {df[df['Tipo'] == 'Tipo C']['Quantidade'].sum()}\033[0m


\033[1mTotal de caixas prontas: {df['Quantidade'].sum()}\033[0m
Data: {datetime.now().strftime("%d-%m-%Y")}

'''
print(Relatorio)








# Relatório Agencia Agteste - CGC: 001

Tipo A: 21
Tipo B: 180
Tipo C: 0


Total de caixas prontas: 201
Data: 06-07-2025




In [ ]:
os.chdir(userdata.get('base'))

In [ ]:
df.to_excel(f'relatorio_Agencia_{nome}_{agencia}_Data_{datetime.now().strftime("%d-%m-%Y")}.xlsx', sheet_name=f'{nome}-{agencia}')